# Agent Demo

In this Demo we are going to show how to use the ReACT agent to answer questions about Iron Maiden.

Goal of this code is to showcase use case of [Intelligent Agents](https://python.langchain.com/docs/modules/agents/) capable of reading Wikipedia articles to properly answer questions about different topics.

Python code bellow makes use of the following software:
 - [langchain](https://www.langchain.com) - a library for building language chains
 - [ctransformers](https://github.com/marella/ctransformers) - c++ hardware (GPU) accelerated transformers library
 - [llama2](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) - llama2 large language model

The agent is a implementation makes use of the [ReAct Prompting](https://react-lm.github.io) algorithm. A ReAct prompt consists of few-shot task-solving trajectories, with human-written text reasoning traces and actions, as well as environment observations in response to actions. 


In [9]:
# imports
import langchain
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.llms import CTransformers

In [10]:
# globals
model_path = "/Users/bsantanna/dev/workspace/community/Llama-2-7b-chat-hf"  # cloned from https://huggingface.co/meta-llama/Llama-2-7b-chat-hf 
instruction_model_path = f"{model_path}/gguf-model-f16.bin"  # converted with https://github.com/ggerganov/llama.cpp 
question = "What is the latest album released by Iron Maiden?"  # example question, a good one :D
n_gpu_layers = 32
n_batch = 512
n_ctx = 5120
n_tokens = 256
n_repetition_penalty = 1.0
n_temperature = 0.6
config = {
    'max_new_tokens': n_tokens,
    'repetition_penalty': n_repetition_penalty,
    'batch_size': n_batch,
    'context_length': n_ctx,
    'reset': False,
    'temperature': n_temperature,
    'gpu_layers': n_gpu_layers
}

## Model loading

In this block model is loaded using optimized parameters for GPU inference.

In [11]:
# load model 
model = CTransformers(model=instruction_model_path, gpu_layers=n_gpu_layers, config=config)

## LLM direct inference
This block shows how to use the pipeline to pass a question directly to the model and get a response. 
It can be observed the model understands and replies the question, however the answer is not up to date with the latest album released by Iron Maiden. It can very well be the case the dataset used to train the model is not up to date.

In [12]:
print(model(question))



Iron Maiden is a British heavy metal band that has been active since 1975. The band has released a total of 16 studio albums, with their latest album being "The Book of Souls" released in 2015.

"The Book of Souls" is a double album that features 11 tracks, including the epic 18-minute long title track. The album was well received by fans and critics alike, and is considered one of the band's best works.

Iron Maiden is known for their powerful and energetic live performances, and they continue to tour and perform live shows to this day. The band has a dedicated fan base around the world, and their music has had a significant influence on the heavy metal genre as a whole.


## ReAct Agent with LLM backend inference

This block shows how to use the pipeline to pass a question to the agent and get a response.

The agent will fork the dialog between two threads and start a problem solving dialog using the ReAct algorithm.

It will first understand the question, search the main concept in Wikipedia, read the articles and then answer the question. 

In [13]:
# load agent with wikipedia
tools = load_tools(["ddg-search", "wikipedia"], llm=model)
PREFIX = """[INST] <<SYS>>
Answer the question delimited by triple backticks as best as you can.
You have access to the following tools: """

SUFFIX = """<</SYS>>
Question: ```{input}```
Thought: {agent_scratchpad}
[/INST]"""
agent = initialize_agent(
    tools,
    model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    agent_kwargs={
        'prefix': PREFIX,
        'suffix': SUFFIX,
    }
)

In [14]:
# result = agent(question)['output']
# print(f"ReACT result:\n\n{result}")

In [15]:
langchain.debug = True
result = agent(question)['output']
langchain.debug = False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the latest album released by Iron Maiden?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is the latest album released by Iron Maiden?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:CTransformers] Entering LLM run with input:
{
  "prompts": [
    "[INST] <<SYS>>\nAnswer the question delimited by triple backticks as best as you can.\nYou have access to the following tools: \n\nduckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.\nWikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\n\nUs

In [16]:
print(f"ReACT result: {result}")

ReACT result: The latest album released by Iron Maiden is "The Mandrake Project", which is expected to be released in 2024.
